In [74]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel
from pgmpy.inference import VariableElimination

In [75]:
wc_model = BayesianModel([('Genetics','Performance'),
                         ('Practise','Performance'),
                         ('Performance','Offer')])

In [76]:
genetics_cpd = TabularCPD(variable = 'Genetics',
                         variable_card = 2,
                         values = [[0.2,0.8]])
# varibale_card = number of outcomes

In [77]:
practise_cpd = TabularCPD(variable = 'Practise',
                         variable_card = 2,
                         values = [[0.7,0.3]])

In [78]:
wc_trials_cpd = TabularCPD(variable = 'Performance',
                      variable_card = 3,
                      values = [[0.5,0.8,0.8,0.9],
                               [0.3,0.15,0.1,0.08],
                               [0.2,0.05,0.1,0.02]],
                      evidence = ['Genetics','Practise'],
                      evidence_card = [2,2])

In [79]:
# evidence == parent
offer_cpd = TabularCPD(variable = 'Offer',
                      variable_card = 2,
                      values = [[0.95,0.8,0.5],
                               [0.05,0.2,0.5]],
                      evidence = ['Performance'],
                      evidence_card = [3])


In [80]:
# Adding relationship between variables
wc_model.add_cpds(genetics_cpd,practise_cpd,wc_trials_cpd,offer_cpd)

In [81]:
# Structure of the above graph
wc_model.get_cpds()

[<TabularCPD representing P(Genetics:2) at 0x2426fdfff08>,
 <TabularCPD representing P(Practise:2) at 0x2426fe55dc8>,
 <TabularCPD representing P(Performance:3 | Genetics:2, Practise:2) at 0x2426fe5f548>,
 <TabularCPD representing P(Offer:2 | Performance:3) at 0x2426fe5f9c8>]

Two nodes (variables) in a Bayes net are on an active trail if a change in one node affects the other.

In [82]:
wc_model.active_trail_nodes('Genetics')

{'Genetics': {'Genetics', 'Offer', 'Performance'}}

In [83]:
wc_model.local_independencies('Genetics')

(Genetics _|_ Practise)

In [84]:
wc_model.get_independencies()

(Genetics _|_ Practise)
(Genetics _|_ Offer | Performance)
(Genetics _|_ Offer | Performance, Practise)
(Practise _|_ Genetics)
(Practise _|_ Offer | Performance)
(Practise _|_ Offer | Performance, Genetics)
(Offer _|_ Genetics, Practise | Performance)
(Offer _|_ Practise | Performance, Genetics)
(Offer _|_ Genetics | Performance, Practise)

In [85]:
wc_infer = VariableElimination(wc_model)

In [87]:
prob_offer = wc_infer.query(variables = ['Offer'])
print(prob_offer)

Eliminating: Practise: 100%|████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 750.59it/s]


+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.8898 |
+----------+--------------+
| Offer(1) |       0.1102 |
+----------+--------------+


In [92]:
prob_offer_good_genes = wc_infer.query(variables = ['Offer'],
                                      evidence = {'Genetics':0})
print(prob_offer_good_genes)

Eliminating: Practise: 100%|████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 569.57it/s]


+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.8420 |
+----------+--------------+
| Offer(1) |       0.1580 |
+----------+--------------+
